# Key Distribution Kata

The **Quantum Key Distribution** kata is a series of exercises designed to teach you about a neat quantum technology where you can use qubits to exchange secure cryptographic keys. In particular, you will work through implementing and testing a quantum key distribution protocol called [BB84](https://en.wikipedia.org/wiki/BB84). 

### Background

What does a key distribution protocol look like in general? Normally there are two parties, commonly referred to as Alice and Bob, who want to share a random, secret string of bits called a _key_. This key can then be used for a variety of different [cryptographic protocols](https://en.wikipedia.org/wiki/Cryptographic_protocol) like encryption or authentication. Quantum versions of key exchange protocols look very similar, and utilize qubits as a way to securely transmit the bit string. 

<img src="./img/qkd-concept.png" alt="General schematic for QKD protocol" width="60%"/>

You can see in the figure above that Alice and Bob have two connections, one quantum channel and one bidirectional classical channel. In this kata you will simulate what happens on the quantum channel by preparing and measuring a sequence of qubits and then perform classical operations to transform the measurement results to a usable, binary key.

There are a variety of different quantum key distribution protocols, however the most common is called [BB84](https://en.wikipedia.org/wiki/BB84) after the initials of the authors and the year it was published. It is used in many existing commercial quantum key distribution devices that implement BB84 with single photons as the qubits. 

#### For more information:
* [Introduction to quantum cryptography and BB84](https://www.youtube.com/watch?v=UiJiXNEm-Go).
* [QKD summer school lecture on quantum key distribution](https://www.youtube.com/watch?v=oEJOtu0joXk).
* [Key Distribution Wikipedia article](https://en.wikipedia.org/wiki/Quantum_key_distribution).
* [BB84 protocol Wikipedia article](https://en.wikipedia.org/wiki/BB84).
* [Updated version of the BB84 paper](https://www.sciencedirect.com/science/article/pii/S0304397514004241?via%3Dihub).

---
### Instructions

Each task is wrapped in one operation preceded by the description of the task.

Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).


## Part I. Preparation

BB84 protocol loops through the two main steps until Alice and Bob have as much key as they want.
The first step is to use the quantum channel, where Alice prepares individual qubits and then sends them to Bob to be measured.
The second step is entirely classical post-processing and communication that takes the measurement results from the quantum step and extracts a classical, random bit string Alice and Bob can use.
Let's start by looking at how Alice will prepare her qubits for sending to Bob as a part of the quantum phase of the BB84 protocol.

Alice has two choices for each qubit, which basis to prepare it in, and what bit value she wants to encode.
This leads to four possible states each qubit can be in that Alice sends out.
The bases she has to choose from are selected such that if an eavesdropper tries to measure a qubit in transit and chooses the wrong basis, then they just get a 0 or 1 with equal probability.

In the first basis, called the computational basis, Alice prepares the states $|0\rangle$ and $|1\rangle$ where $|0\rangle$ represents the key bit value `0` and $|1\rangle$ represents the key bit value `1`.
The second basis (sometimes called the diagonal or Hadamard basis) uses the states $|+\rangle = \frac{1}{\sqrt2}(|0\rangle + |1\rangle)$ to represent the key bit value `0`, and $|-\rangle = \frac{1}{\sqrt2}(|0\rangle - |1\rangle)$ to represent the key bit value `1`.


### <a name="Task-1.1-diagonal-basis"></a> Task 1.1. Diagonal basis

Try your hand at converting qubits from the computational basis to the diagonal basis.

**Input:** $N$ qubits (stored in an array of length $N$). Each qubit is either in $|0\rangle$ or in $|1\rangle$ state.

**Goal:**  Convert the qubits to the diagonal basis: 
* if `qs[i]` was in state $|0\rangle$, it should be transformed to $|+\rangle = \frac{1}{\sqrt2}(|0\rangle + |1\rangle)$,
* if `qs[i]` was in state $|1\rangle$, it should be transformed to $|-\rangle = \frac{1}{\sqrt2}(|0\rangle - |1\rangle)$.

In [1]:
%kata T11_DiagonalBasis

operation DiagonalBasis (qs : Qubit[]) : Unit {    
    ApplyToEach(H, qs);
}

Success!

*Can't come up with a solution? See the explained solution in the [Key Distribution - BB84 Workbook](./Workbook_KeyDistribution_BB84.ipynb#diagonal_basis).*

### <a name="Task-1.2-equal-superposition"></a> Task 1.2. Equal superposition

 
**Input**: A qubit in the $|0\rangle$ state.

**Goal**:  Change the qubit state to a superposition state that has equal probabilities of measuring 0 and 1. 

> Note that this is not the same as keeping the qubit in the $|0\rangle$ state with 50% probability and converting it to the $|1\rangle$ state with 50% probability!

In [2]:
%kata T12_EqualSuperposition 

operation EqualSuperposition (q : Qubit) : Unit {
    H(q);
}

|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖

Success!

*Can't come up with a solution? See the explained solution in the [Key Distribution - BB84 Workbook](./Workbook_KeyDistribution_BB84.ipynb#equal_superposition).*

## Part II. BB84 Protocol

Now that you have seen some of the steps that Alice will use to prepare here qubits, it's time to add in the rest of the steps of the BB84 protocol.

### <a name="Task-2.1-generate-random-array"></a>Task 2.1. Generate random array

You saw in part I that Alice has to make two random choices per qubit she prepares, one for which basis to prepare in, and the other for what bit value she wants to send.
Bob will also need one random bit value to decide what basis he will be measuring each qubit in.
To make this easier for later steps, you will need a way of generating random boolean values for both Alice and Bob to use.

**Input:** An integer $N$.

**Output** :  A `Bool` array of length N, where each element is chosen at random. 

> This will be used by both Alice and Bob to choose either the sequence of bits to send or the sequence of bases (`false` indicates $|0\rangle$ / $|1\rangle$ basis, and `true` indicates $|+\rangle$ / $|-\rangle$ basis) to use when encoding/measuring the bits.

In [3]:
%kata T21_RandomArray 

// Required to use DrawRandomBool function
open Microsoft.Quantum.Random;

operation RandomArray (N : Int) : Bool[] {
    // Step 1: Create array of size N with default value false
    mutable array = [false, size = N];

    // Step 2: Iterate through all elements of the array and set the random value using DrawRandomBool function
    for i in 0 .. N - 1 {
        set array w/= i <- DrawRandomBool(0.5);
    }

    // Step 3: Return the random bool array
    return array;
}

Success!

*Can't come up with a solution? See the explained solution in the [Key Distribution - BB84 Workbook](./Workbook_KeyDistribution_BB84.ipynb#generate_random_array).*

### <a name="Task-2.2-prepare-alice-qubits"></a>Task 2.2. Prepare Alice's qubits

Now that you have a way of generating the random inputs needed for Alice and Bob, it's time for Alice to use the random bits to prepare her sequence of qubits to send to Bob.

**Inputs:** 

1. `qs`: an array of $N$ qubits in the $|0\rangle$ states,
2. `bases`: a `Bool` array of length $N$; 
    `bases[i]` indicates the basis to prepare the i-th qubit in:  
    * `false`: use $|0\rangle$ / $|1\rangle$ (computational) basis,
    * `true`: use $|+\rangle$ / $|-\rangle$ (Hadamard/diagonal) basis.
3. `bits`: a `Bool` array of length $N$;
    `bits[i]` indicates the bit to encode in the i-th qubit: `false` = 0, `true` = 1.

**Goal:**  Prepare the qubits in the described state.

In [4]:
%kata T22_PrepareAlicesQubits

operation PrepareAlicesQubits (qs : Qubit[], bases : Bool[], bits : Bool[]) : Unit {
    for i in 0 .. Length(qs) - 1 {
        if (bits[i]) {
            X(qs[i]);
        }
        
        if (bases[i]) {
            // use Hadamard/diagonal basis.
            H(qs[i]);
        }
    }
}

Success!

*Can't come up with a solution? See the explained solution in the [Key Distribution - BB84 Workbook](./Workbook_KeyDistribution_BB84.ipynb#prepare_alice_qubits).*

### <a name="Task-2.3-measure-bob-qubits"></a>Task 2.3. Measure Bob's qubits

Bob now has an incoming stream of qubits that he needs to measure by randomly choosing a basis to measure in for each qubit.

**Inputs:**

1. `qs`: an array of $N$ qubits;  
   each qubit is in one of the following states: $|0\rangle$, $|1\rangle$, $|+\rangle$, $|-\rangle$. 
2. `bases`: a `Bool` array of length $N$; 
   `bases[i]` indicates the basis used to prepare the i-th qubit:
   * `false`: $|0\rangle$ / $|1\rangle$ (computational) basis,
   * `true`: $|+\rangle$ / $|-\rangle$ (Hadamard/diagonal) basis.

**Output:** Measure each qubit in the corresponding basis and return an array of results as boolean values, encoding measurement result `Zero` as `false` and `One` as `true`. 
The state of the qubits at the end of the operation does not matter.

In [5]:
%kata T23_MeasureBobsQubits

operation MeasureBobsQubits (qs : Qubit[], bases : Bool[]) : Bool[] {
    let N = Length(qs);
    mutable array = [false, size = N];

    for i in 0 .. N - 1 {
        if (bases[i]) {
            H(qs[i]);
        }
        
        set array w/= i <- M(qs[i]) == One;
    }

    return array;
}

Success!

*Can't come up with a solution? See the explained solution in the [Key Distribution - BB84 Workbook](./Workbook_KeyDistribution_BB84.ipynb#measure_bob_qubits).*

### <a name="Task-2.4-generate-shared-key"></a> Task 2.4. Generate the shared key!

Now, Alice has a list of the bit values she sent as well as what basis she prepared each qubit in, and Bob has a list of bases he used to measure each qubit. To figure out the shared key, they need to figure out when they both used the same basis, and toss the data from qubits where they used different bases. If Alice and Bob did not use the same basis to prepare and measure the qubits in, the measurement results Bob got will be just random with 50% probability for both the `Zero` and `One` outcomes.
    
**Inputs:**

1. `basesAlice` and `basesBob`: `Bool` arrays of length $N$
   describing Alice's and Bobs's choice of bases, respectively;
2. `measurementsBob`: a `Bool` array of length $N$ describing Bob's measurement results.
    
**Output:** a `Bool` array representing the shared key generated by the protocol.

> Note that you don't need to know both Alice's and Bob's bits to figure out the shared key!

In [9]:
%kata T24_GenerateSharedKey

function GenerateSharedKey (basesAlice : Bool[], basesBob : Bool[], measurementsBob : Bool[]) : Bool[] {
    let N = Length(measurementsBob);
    mutable array = [];
    for i in 0 .. N - 1 {
        if (basesAlice[i] == basesBob[i]) {
            set array = array + [measurementsBob[i]];
        }
    }
    
    return array;
}

Success!

*Can't come up with a solution? See the explained solution in the [Key Distribution - BB84 Workbook](./Workbook_KeyDistribution_BB84.ipynb#generate_the_shared_key).*

### <a name="Task-2.5-check-error-rate"></a> Task 2.5. Check if error rate was low enough

The main trace eavesdroppers can leave on a key exchange is to introduce more errors into the transmission. Alice and Bob should have characterized the error rate of their channel before launching the protocol, and need to make sure when exchanging the key that there were not more errors than they expected. The "errorRate" parameter represents their earlier characterization of their channel.

**Inputs:**

1. `keyAlice` and `keyBob`: `Bool` arrays of equal length $N$ describing 
   the versions of the shared key obtained by Alice and Bob, respectively.
2. `errorRate`: an integer between 0 and 50 - the percentage of the bits that did not match in Alice's and Bob's channel characterization.
    
**Output:** `true` if the percentage of errors is less than or equal to the error rate, and `false` otherwise.

In [25]:
%kata T25_CheckKeysMatch

open Microsoft.Quantum.Convert;

function CheckKeysMatch (keyAlice : Bool[], keyBob : Bool[], errorRate : Int) : Bool {
    mutable errorCount = 0;
    let N = Length(keyAlice);
    for i in 0 .. N - 1 {
        if (keyAlice[i] != keyBob[i]) {
            set errorCount = errorCount + 1;
        }
    }
    
    let percentage = IntAsDouble(errorCount) / IntAsDouble(N);
    return percentage <= IntAsDouble(errorRate) / 100.0;
}

Success!

*Can't come up with a solution? See the explained solution in the [Key Distribution - BB84 Workbook](./Workbook_KeyDistribution_BB84.ipynb#check_error_rate).*

### <a name="Task-2.6-putting-it-all-together"></a> Task 2.6. Putting it all together

**Goal:** Implement the entire BB84 protocol using tasks 2.1 - 2.5 
and following the comments in the operation template. 

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_BB84Protocol` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_BB84Protocol`).

In [31]:
operation Run_BB84Protocol () : Unit {
     // Require a nearly perfect match of the shared keys for success
    // (1% of the characters are allowed to be different).
    let threshold = 1;

    use qs = Qubit[20];
    // 1. Choose random basis and bits to encode
    let basesAlice = RandomArray(Length(qs));
    let bitsAlice = RandomArray(Length(qs));

    // 2. Alice prepares her qubits
    PrepareAlicesQubits(qs, basesAlice, bitsAlice);

    // 3. Bob chooses random basis to measure in
    let basesBob = RandomArray(Length(qs));

    // 4. Bob measures Alice's qubits
    let bitsBob = MeasureBobsQubits(qs, basesBob);

    // 5. Generate shared key
    let keyAlice = GenerateSharedKey(basesAlice, basesBob, bitsAlice);
    let keyBob = GenerateSharedKey(basesAlice, basesBob, bitsBob);

    // 6. Ensure at least the minimum percentage of bits match
    if (CheckKeysMatch(keyAlice, keyBob, threshold)) {
        Message($"Successfully generated keys {keyAlice}/{keyBob}");
    }
}

Run_BB84Protocol

In [32]:
%simulate Run_BB84Protocol

Successfully generated keys [True,True,False,True,True,False,True,True]/[True,True,False,True,True,False,True,True]


()

*Can't come up with a solution? See the explained solution in the [Key Distribution - BB84 Workbook](./Workbook_KeyDistribution_BB84.ipynb#putting_it_all_together).*

## Part III. Eavesdropping

### <a name="Task-3.1-eavesdrop"></a>Task 3.1. Eavesdrop!

In this task you will try to implement an eavesdropper, Eve. 

Eve will intercept a qubit from the quantum channel that Alice and Bob are using. 
She will measure it in either the $|0\rangle$ / $|1\rangle$ basis or the $|+\rangle$ / $|-\rangle$ basis, and prepare a new qubit in the state she measured. Then she will send the new qubit back to the channel. 
Eve hopes that if she got lucky with her measurement, that when Bob measures the qubit he doesn't get an error so she won't be caught!

**Inputs:**

1. `q`: a qubit in one of the following states: $|0\rangle$, $|1\rangle$, $|+\rangle$, $|-\rangle$.
2. `basis`: Eve's guess of the basis she should use for measuring.
   Recall that `false` indicates $|0\rangle$ / $|1\rangle$ basis and `true` indicates $|+\rangle$ / $|-\rangle$ basis. 

**Output:** the bit encoded in the qubit (`false` for $|0\rangle$ / $|+\rangle$ states, `true` for $|1\rangle$ / $|-\rangle$ states).

   > In this task you are guaranteed that the basis you're given matches the one in which the qubit is encoded, that is, if you are given a qubit in state $|0\rangle$ or $|1\rangle$, you will be given `basis = false`, and if you are given a qubit in state $|+\rangle$ or $|-\rangle$, you will be given `basis = true`. This is different from a real eavesdropping scenario, in which you have to guess the basis yourself.

In [34]:
%kata T31_Eavesdrop

operation Eavesdrop (q : Qubit, basis : Bool) : Bool {
    // Measurement along X axis if basis is diagonal basis and Z axis, otherwise.
    return ResultAsBool(Measure([basis ? PauliX | PauliZ], [q]));
}

Success!

*Can't come up with a solution? See the explained solution in the [Key Distribution - BB84 Workbook](./Workbook_KeyDistribution_BB84.ipynb#eavesdrop).*

### <a name="Task-3.2-catch-the-eavesdropper"></a>Task 3.2. Catch the eavesdropper

Add an eavesdropper into the BB84 protocol from task 2.6. 

Note that now we should be able to detect Eve and therefore we have to discard our key bits!

> Similar to task 2.6, this is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_BB84ProtocolWithEavesdropper` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_BB84ProtocolWithEavesdropper`).

In [35]:
operation Run_BB84ProtocolWithEavesdropper () : Unit {
    let threshold = 1;

    use qs = Qubit[20];
    // 1. Choose random basis and bits to encode
    let basesAlice = RandomArray(Length(qs));
    let bitsAlice = RandomArray(Length(qs));

    // 2. Alice prepares her qubits
    PrepareAlicesQubits(qs, basesAlice, bitsAlice);

    // Eve eavesdrops on all qubits, guessing the basis at random
    for q in qs {
        let n = Eavesdrop(q, DrawRandomBool(0.5));
    }

    // 3. Bob chooses random basis to measure in
    let basesBob = RandomArray(Length(qs));

    // 4. Bob measures Alice's qubits'
    let bitsBob = MeasureBobsQubits(qs, basesBob);

    // 5. Generate shared key
    let keyAlice = GenerateSharedKey(basesAlice, basesBob, bitsAlice);
    let keyBob = GenerateSharedKey(basesAlice, basesBob, bitsBob);

    // 6. Ensure at least the minimum percentage of bits match
    if (CheckKeysMatch(keyAlice, keyBob, threshold)) {
        Message($"Successfully generated keys {keyAlice}/{keyBob}");
    } else {
        Message($"Caught an eavesdropper, discarding the keys {keyAlice}/{keyBob}");
    }
}

Run_BB84ProtocolWithEavesdropper

In [36]:
%simulate Run_BB84ProtocolWithEavesdropper

Caught an eavesdropper, discarding the keys [True,False,False,True,False,False,False,True,True]/[False,False,False,True,False,False,False,False,False]


()

*Can't come up with a solution? See the explained solution in the [Key Distribution - BB84 Workbook](./Workbook_KeyDistribution_BB84.ipynb#catch_the_eavesdropper).*